In [3]:
import pandas as pd
import json
from sbscorer.sbdata.FlipsideApi import FlipsideApi
import os
import numpy as np
from pathlib import Path

import  json

In [5]:
FOLDER_NAME = 'cgr'
CHAIN = 'ethereum'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)

# Initialize Flipside API
api_key = os.environ['FLIPSIDE_API_KEY']
flipside_api = FlipsideApi(api_key, max_address=100)

In [8]:
PATH_TO_VOTES = os.path.join(current_dir.parent.parent.parent, 'cgr2', 'votes_with_time.csv')

df_votes = pd.read_csv(PATH_TO_VOTES)
json_votes = df_votes.to_json(orient='records')
parsed_json = json.loads(json_votes)
escaped_json_string = json.dumps(parsed_json)
escaped_json_string

'[{"id": "0xea21b4ad8fb8a7dbebaf604a59da47c9a12f36a21a80227d00706f75ea48a1e2", "transaction": "0x1fccac24035b30a805abc4654ea58b9ff784e70f40b82cccb0f9185148fec306", "blockNumber": 3895239, "projectId": "0x769847f797115434861cd3c1ba7ae09cde925500f55bc74a405f0008df0d972a", "applicationId": 47, "roundId": "0x7492a8c4ED29B1F1559888Bd832fAe5D33E10370", "voter": "0xfdb1517AC55F9285Ed13A72C9c872c7b3b0f57a8", "grantAddress": "0x2dd2036C9Db2ADA2739509AF0047c00C8b9291EF", "token": "0x0000000000000000000000000000000000000000", "amount": 2000000000000000, "amountUSD": 3.16035552, "amountRoundToken": 3.61351873e+18, "blockTime": "2023-10-10T00:20:05.000000Z", "unixTime": 1696897205}, {"id": "0x7067a5fe1c22677a273628befb90655e750f0999888311b17c22096cbf471a71", "transaction": "0x1fccac24035b30a805abc4654ea58b9ff784e70f40b82cccb0f9185148fec306", "blockNumber": 3895239, "projectId": "0x81ffd035aaba40a60d0a6651ff257fe7738e2f65eb47d59ae32fe84593f6ab01", "applicationId": 34, "roundId": "0x7492a8c4ED29B1F15

In [10]:
sql_template = """

with vote_query AS (
SELECT 
  value[0]::STRING AS id,
  value[1]::STRING AS transaction,
  value[2]::NUMBER AS block_number,
  value[3]::STRING AS project_id,
  value[4]::NUMBER AS application_id,
  value[5]::STRING AS round_id,
  value[6]::STRING AS voter,
  value[7]::STRING AS grant_address,
  value[8]::STRING AS token,
  value[9]::NUMBER AS amount,
  value[10]::FLOAT AS amount_USD,
  value[11]::FLOAT AS amount_round_token,
  value[12]::TIMESTAMP_NTZ AS block_time,
  value[13]::NUMBER AS unix_time
FROM (
  SELECT
    livequery.live.udf_api('https://flipsidecrypto.xyz/api/queries/cb6373b7-9e55-4df4-9643-6117da959924/latest-run') as response
  ), lateral FLATTEN (input => response:data:data)
),
    unique_voter as (SELECT DISTINCT voter from vote_query),

    SELECT 
    tx.TX_HASH,
    tx.BLOCK_TIMESTAMP,
    tx.BLOCK_NUMBER,
    tx.FROM_ADDRESS,
    tx.TO_ADDRESS,
    tx.GAS_LIMIT,
    tx.GAS_USED,
    tx.TX_FEE,
    tx.ETH_VALUE
    FROM %s.core.fact_transactions tx
    INNER JOIN
    unique_voter v ON tx.FROM_ADDRESS = v.voter OR tx.TO_ADDRESS = v.voter
    WHERE tx.BLOCK_TIMESTAMP 
    between CURRENT_DATE - interval '%d day'
    AND CURRENT_DATE - interval '%d day'
    AND tx.STATUS = 'SUCCESS'  
    """

In [11]:
N_DAYS = 1
i = 0
sql = sql_template % (CHAIN, (i+1)*N_DAYS, i*N_DAYS)

In [5]:
query_result_set = flipside_api.sdk.query(sql_template,
                                              max_age_minutes=flipside_api.MAX_AGE_MINUTES,
                                              page_size=50000,
                                              page_number=1,
                                              timeout_minutes=flipside_api.TIMEOUT_MINUTES,
                                              ttl_minutes=flipside_api.TTL_MINUTES,
                                              cached=flipside_api.CACHED,
                                              retry_interval_seconds=flipside_api.RETRY_INTERVAL_SECONDS)

In [6]:
def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return f"{size:.{decimal_places}f} {unit}"
        size /= 1024.0
    return f"{size:.{decimal_places}f} PB"

print(human_readable_size(query_result_set.run_stats.bytes))


23.27 MB
